# Advent of Code 2022

## Day 20: Grove Positioning System

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

I solved day 20 pretty quickly, but I'm writing this with day 19 part 2 from yesterday still unsolved.

I started the solution for part 1 by making a data structure I called `Ring` which stores `int` objects inside a data structure called `LinkedNode`. A `dict` stores pointers to the nodes to jump immediately to each node for only the cost of a `dict` lookup, and moving 1 space left of right takes constant time. When moving `n` spaces, only one lookup is done and the whole operation is `O(n)`. So moving `m` ints takes `O(m*max(n)`. I added a `__getitem__` and `__len__` to `Ring` to read it back like a list, with modular arithmetic applied to the index so solution is `ring[1000] + ring[2000] + ring[3000]`. However, if the ring isn't mutated between reads I need to cache the list to avoid recomputing it each time, so subsequent reads are constant time.

However, my input file contains repeated values! Unlike the example. The fix was not difficult, I just replaced `int` with an object I call `ReferenceInt`. It acts like an `int` but it's equality and hash code is based on object identity not value., and part 1 is solved.

For part 2 I had to divide all ints through by `m-1` and take the remainder. I used my `ReferenceInt` to store both the original and remainder value. Since the

### Imports

In [ ]:
from typing import Optional
import tqdm.notebook as tqdm
from typing import Iterator

### Reference Int

In [ ]:
class ReferenceInt:

    __next_hash = 0

    __slots__ = ['remainder', 'original_value', 'hash_proxy']

    def __init__(self, original_value : int):
        self.remainder = original_value
        self.original_value = original_value
        self.hash_proxy = ReferenceInt.__next_hash
        ReferenceInt.__next_hash += 1

    def divide(self, base: int):
        assert base >= 1
        if self.original_value > 0:
            self.remainder = self.remainder % base
        else:
            self.remainder = -((-self.remainder) % base)

    def __str__(self):
        if self.remainder == self.original_value:
            return f'<{self.original_value}>'
        return f'<{self.remainder}|{self.original_value}>'

    def __repr__(self):
        return str(self)

    def __eq__(self, other) -> bool:
        return self is other

    def __ne__(self, other: 'ReferenceInt') -> bool:
        return self is not other

    def __hash__(self):
        return hash(self.hash_proxy)

### Linked Node

In [ ]:
class LinkedNode:

    __slots__ = ['previous_node', 'next_node', 'value']

    def __init__(self, value: ReferenceInt):
        self.previous_node: Optional[LinkedNode] = None
        self.next_node: Optional[LinkedNode] = None
        self.value = value

    def link(self, previous_node: 'LinkedNode', next_node: 'LinkedNode') -> None:
        self.previous_node = previous_node
        self.next_node = next_node

    @staticmethod
    def relink(a: 'LinkedNode', b: 'LinkedNode', c: 'LinkedNode', d: 'LinkedNode'):
        a.next_node = b
        b.next_node = c
        c.next_node = d
        d.previous_node = c
        c.previous_node = b
        b.previous_node = a

    def move_right(self) -> None:
        LinkedNode.relink(self.previous_node, self.next_node, self, self.next_node.next_node)

    def move_left(self) -> None:
        LinkedNode.relink(self.previous_node.previous_node, self, self.previous_node, self.next_node)

    def __str__(self):
        return str(self.value)

    def __repr__(self):
        return str(self)

### Ring

In [ ]:
class Ring:

    __slots__ = ['codes', 'nodes', 'flattened', 'start_value']

    def __init__(self, arrangement: list[ReferenceInt], start_value: ReferenceInt):

        self.codes = arrangement
        self.nodes: dict[ReferenceInt, LinkedNode] = {value: LinkedNode(value) for value in arrangement}
        self.flattened: Optional[list[int]] = None
        self.start_value = start_value

        for listIndex, value in enumerate(arrangement):

            previousIndex: int = (listIndex - 1) % len(arrangement)
            nextIndex: int = (listIndex + 1) % len(arrangement)

            previousValue: ReferenceInt = arrangement[previousIndex]
            nextValue: ReferenceInt = arrangement[nextIndex]

            previous_node: LinkedNode = self.nodes[previousValue]
            next_node: LinkedNode = self.nodes[nextValue]
            node: LinkedNode = self.nodes[value]

            node.link(previous_node, next_node)

    def __len__(self):
        return len(self.nodes)

    def __getitem__(self, item: int) -> int:
        assert item >= 0
        if self.flattened is None:
            self.flattened = self.to_list()
        return self.flattened[item % len(self.flattened)]

    def mix(self, value: ReferenceInt) -> None:
        self.flattened = None
        node = self.nodes[value]
        if value.remainder < 0:
            for _ in range(-value.remainder):
                node.move_left()
        else:
            for _ in range(value.remainder):
                node.move_right()

    def values(self) -> Iterator[int]:
        yield self.start_value
        node: LinkedNode = self.nodes[self.start_value].next_node
        while node.value != self.start_value:
            yield node.value
            node = node.next_node

    def to_list(self) -> list[int]:
        return [value.original_value for value in tqdm.tqdm(self.values(), total=len(self), desc='flattening list')]

    def __str__(self):
        node: LinkedNode = self.nodes[self.start_value]
        rv = str(node)
        node = node.next_node
        while node.value != self.start_value:
            rv += ' -> ' + str(node)
            node = node.next_node
        rv += ' -> ' + str(self.start_value)
        return rv

    def __repr__(self):
        return str(self)

### Input Parsing

In [ ]:
INPUT_FILE = 'data/input20.txt'

In [ ]:
def read_ints(filename: str, key: int = 1) -> Iterator[ReferenceInt]:
    with open(filename) as file:
        for line in file:
            line = line.strip()
            current = ReferenceInt(int(line) * key)
            yield current

In [ ]:
def read_codes(filename: str, key: int = 1) -> Ring:
    codes = []
    start_value = None
    for current in read_ints(filename, key):
        if current.original_value == 0:
            start_value = current
        codes.append(current)
    assert start_value is not None
    return Ring(codes, start_value)

### Part 1

In [ ]:
def main():
    ring = read_codes(INPUT_FILE)
    for c in tqdm.tqdm(ring.codes, desc='mixing ring'):
        ring.mix(c)
    print(f'The sum of the three numbers that form the grove coordinates is {ring[1000] + ring[2000] + ring[3000]}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
TIMES_TO_MIX = 10
KEY = 811589153

In [ ]:
def main():
    ring = read_codes(INPUT_FILE, KEY)
    for value in ring.codes:
        value.divide(len(ring) - 1)
    for c in tqdm.tqdm(ring.codes * TIMES_TO_MIX, desc='mixing ring'):
        ring.mix(c)
    print(f'The sum of the three numbers that form the grove coordinates is {ring[1000] + ring[2000] + ring[3000]}')

In [ ]:
if __name__ == '__main__':
    main()